In [1]:
#!/usr/bin/env python
# coding=utf-8
# 《省平台调度每周新冠接种情况》.xlsx
# author: KarlQu
# Version: 1.0
# Date: 2022-09-28
# requests.__version__ = '2.24.0'
# webdriver.__version__ = '3.14.1'
# pd.__version__ = '1.1.3'
# 登录VPN后，运行该文件，等待selenium弹出网页输入验证码即可。
# 完成后找到result.csv文件，ctrl+c复制内容，ctrl+shift+v仅复制数值，而后微调两个接种覆盖率数据。

import requests
import time
import random
import os
import pickle
import pandas as pd
import calendar
from datetime import datetime, timedelta
from selenium import webdriver
from datetime import datetime, timedelta
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By


class PostTables():
    """生成请求表单
    """
    def __init__(self) -> None:
        """
        自动：根据现在日期，准备'datebean.endDate':date,'datebean.beginDate':weekago
        #! 因为表3-3-2和3-3-4和4-3-3具有时效性，无法选择截止日期，因此只能选择周一使用
        Parameters
        ----------
        # half_year : str
        #     需手动传入：提供半年前日期的字符串，月底对应月底；唯一接口
        # ! 已经写好自动生成函数了，在self.half_year_ago()
        """
        self.date = str(datetime.date(datetime.now()) - timedelta(1))  # 减一天
        self.weekago = str(datetime.date(datetime.now()) - timedelta(8))  # 减一周
        self.half_year = self.half_year_ago(
            datetime.date(datetime.now()) - timedelta(1))
        self.cmodid_to_url = {
            '1010011123':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptArea_query.action',
            '1010011142':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptJqmyArea_query.action',
            '1010011140':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptJqmy_query.action',
            '1010011135':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptWqjz_query.action'
        }
        self.used_tables = {
            '1-1-1': '分地区接种信息汇总表，第一版，beginDate:weekago',
            '1-2-1': '分地区接种信息汇总表，第二版，60岁以上第一针覆盖率，实种需调整，应种七普加黄',
            '1-2-2': '分地区接种信息汇总表，第二版，60-69岁第一针覆盖率，实种，应种七普加黄',
            '1-2-3': '分地区接种信息汇总表，第二版，70-79岁第一针覆盖率，实种，应种七普加黄',
            '1-2-4': '分地区接种信息汇总表，第二版，80岁以上第一针覆盖率，实种需调整，应种七普加黄',
            '1-3-1-1': '分地区接种信息汇总表，第三版，加强的应种，到期（截止半年前的日期，月底到月底）的第二剂，18岁以上数',
            '1-3-1-2':
            '分地区接种信息汇总表，第三版，加强的应种，减去（北京智飞绿竹、康熙诺生物、安徽智飞）三家的第二剂',  # !选厂家，表单提交参数多一项
            '1-3-1-3': '分地区接种信息汇总表，第三版，加强的应种，加上康熙诺生物第一剂',
            '2-3-1-1': '分地区加强接种情况统计表，第三版，加强的实种，全年龄，Vero细胞第三剂累计+腺病毒第二剂累计',
            '3-3-2': '加强免疫统计表，第三版，18岁以上，取数直接计算',
            '3-3-4': '加强免疫统计表，第三版，60岁以上，取数直接计算',  # !ageSel为重要参数，而非age
            '4-3-3': '全程接种统计表，第三版，60岁以上，取数直接计算'
        }
        self.url = {
            'url_1':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptArea_query.action',  # 分地区接种信息汇总表
            'url_2':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptJqmyArea_query.action',  # 分地区加强接种情况统计表
            'url_3':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptJqmy_query.action',  # 加强免疫统计表
            'url_4':
            'http://10.60.0.16:83/ShanDong/gjdc/xgRptWqjz_query.action'  # 全程接种统计表
        }
        self.gen_tables = self.generate()

    def half_year_ago(self, date: datetime.date) -> str:
        """半年前日期，月底对应月底
        Parameters
        ----------
        date : datetime.date
            上周末日期
        Returns
        -------
        str
        """
        month = date.month - 6
        year = int(date.year + month / 12)
        day = min(date.day, calendar.monthrange(year, month)[1])
        dt = date.replace(year=year, month=month, day=day)
        # print(dt)
        dt = str(dt)
        return dt

    def generate(self) -> dict:
        """生成12个表，组成字典返回
        Parameters
        ----------
        half_year : str
            手动输入半年前的日期字符串，月底对应月底即可 例: '2022-3-24'
        Returns
        -------
        dict
            由数字字符串做key索引，格式为'请求表，版面，版内指标，具体表单'。
        """
        tables = {}
        tables.update({
            '1-1-1': {
                'skipValue': '4',
                'areaCode':
                '371082010100,371082010200,371082010300,371082010400,371082010500,371082030100,371082030200,371082038100,371082038200,371082050100,371082050200,371082058100,371082058200,371082070100,371082078100,371082080100,371082088100,371082090100,371082098100,371082100100,371082108100,371082110100,371082118100,371082130100,371082138100,371082140100,371082148100,371082180100,371082180200,371082188100,371082188200,371082190100,371082190200,371082190300,371082190400,371082190500,371082250100,371082258100,371082270100,371082270200,371082278100,371082278200,371082280100,371082280400,371082288100,371082288200,371082298100,371082304100,371082308100,371082318100,371082328100,371082338100,371082340300,371082358100,371082368100,371082374100,371082378100,371082378200',
                'datebean.beginDate': self.weekago,
                'datebean.endDate': self.date,
                '__checkbox_chkFactory': 'true',  # 似乎不重要
                'factoryCode': '02,04,10,11,12,13,14,36,57,66,68,80,81,82,88',
                '__checkbox_chkCrowdkind': 'true',  # 似乎不重要
                'crowdKindCode':
                '01,02,03,04,05,06,07,08,09,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30',
                'chkAge': 'true',  # 重要
                '__checkbox_chkAge': 'true',  # 似乎不重要
                'age': '0,3,6,12,15,18,50,55,60,65,70,80',
                '__multiselect_regtype': '',
                'fsource': '-1',
                'fidCardType': ''
            }
        })
        tables['1-1-1'].update({
            'cModId': '1010011123',
            'cOption': '4'
        })  # 添加表1-的ID
        tables.update({'1-2-1': tables['1-1-1'].copy()})
        tables['1-2-1'].update({
            'datebean.beginDate': '2019-10-01',
            'age': '60,65,70,80'
        })
        tables.update({'1-2-2': tables['1-2-1'].copy()})
        tables['1-2-2'].update({'age': '60,65'})
        tables.update({'1-2-3': tables['1-2-2'].copy()})
        tables['1-2-3'].update({'age': '70'})
        tables.update({'1-2-4': tables['1-2-3'].copy()})
        tables['1-2-4'].update({'age': '80'})
        tables.update({'1-3-1-1': tables['1-2-4'].copy()})
        if not isinstance(self.half_year, str):
            raise TypeError(str)
        tables['1-3-1-1'].update({
            'datebean.endDate': self.half_year,
            'age': '18,50,55,60,65,70,80'
        })
        tables.update({'1-3-1-2': tables['1-1-1'].copy()})
        tables['1-3-1-2'].update({
            'datebean.beginDate': '2019-10-01',
            'chkFactory': 'true',
            'factoryCode': '68,81,82',
            'factoryCodeSel': '68,81,82'
        })
        tables.update({'1-3-1-3': tables['1-3-1-2'].copy()})
        tables['1-3-1-3'].update({'factoryCode': '81', 'factoryCodeSel': '81'})
        tables.update({
            '2-3-1-1': {
                'cModId': '1010011142',
                'cOption': '4',
                'skipValue': '4',
                'areaCode':
                '371082010100,371082010200,371082010300,371082010400,371082010500,371082030100,371082030200,371082038100,371082038200,371082050100,371082050200,371082058100,371082058200,371082070100,371082078100,371082080100,371082088100,371082090100,371082098100,371082100100,371082108100,371082110100,371082118100,371082130100,371082138100,371082140100,371082148100,371082180100,371082180200,371082188100,371082188200,371082190100,371082190200,371082190300,371082190400,371082190500,371082250100,371082258100,371082270100,371082270200,371082278100,371082278200,371082280100,371082280400,371082288100,371082288200,371082298100,371082304100,371082308100,371082318100,371082328100,371082338100,371082340300,371082358100,371082368100,371082374100,371082378100,371082378200',
                'datebean.beginDate': '2019-10-01',
                'datebean.endDate': self.date,
                '__checkbox_chkFactory': 'true',
                'factoryCode': '02,04,10,11,12,13,14,36,57,66,68,80,81,82,88',
                '__checkbox_chkAge': 'true',
                'age': '0,3,6,12,15,18,50,55,60,65,70,80',
                '__multiselect_regtype': ''
            }
        })
        tables.update({
            '3-3-2': {
                'cModId': '1010011140',
                'cOption': '4',
                'skipValue': '4',
                'areaCode':
                '371082010100,371082010200,371082010300,371082010400,371082010500,371082030100,371082030200,371082038100,371082038200,371082050100,371082050200,371082058100,371082058200,371082070100,371082078100,371082080100,371082088100,371082090100,371082098100,371082100100,371082108100,371082110100,371082118100,371082130100,371082138100,371082140100,371082148100,371082180100,371082180200,371082188100,371082188200,371082190100,371082190200,371082190300,371082190400,371082190500,371082250100,371082258100,371082270100,371082270200,371082278100,371082278200,371082280100,371082280400,371082288100,371082288200,371082298100,371082304100,371082308100,371082318100,371082328100,371082338100,371082340300,371082358100,371082368100,371082374100,371082378100,371082378200',
                'chkAge': 'true',
                '__checkbox_chkAge': 'true',
                'ageSel': '18,50,55,60,65,70,80',
                '__multiselect_regtype': ''
            }
        })
        tables.update({'3-3-4': tables['3-3-2'].copy()})
        tables['3-3-4'].update({'ageSel': '60,65,70,80'})
        tables.update({
            '4-3-3': {
                'cModId': '1010011135',
                'cOption': '4',
                'skipValue': '4',
                'areaCode':
                '371082010100,371082010200,371082010300,371082010400,371082010500,371082030100,371082030200,371082038100,371082038200,371082050100,371082050200,371082058100,371082058200,371082070100,371082078100,371082080100,371082088100,371082090100,371082098100,371082100100,371082108100,371082110100,371082118100,371082130100,371082138100,371082140100,371082148100,371082180100,371082180200,371082188100,371082188200,371082190100,371082190200,371082190300,371082190400,371082190500,371082250100,371082258100,371082270100,371082270200,371082278100,371082278200,371082280100,371082280400,371082288100,371082288200,371082298100,371082304100,371082308100,371082318100,371082328100,371082338100,371082340300,371082358100,371082368100,371082374100,371082378100,371082378200',
                'chkAge': 'true',
                '__checkbox_chkAge': 'true',
                'ageSel': '60,65,70,80',
                '__multiselect_regtype': ''
            }
        })

        return tables


# 制作一个门诊归类后取列
class TablesShift():
    def __init__(self) -> None:
        self.zhenjie_mapping = {
            '系统单位': {
                0: '崖头镇寻山卫生院预防接种门诊 3710820101',
                1: '崖头镇崂山卫生院预防接种门诊 3710820102',
                2: '中医院预防接种门诊       3710820103',
                3: '荣成市康宁医院预防接种门诊 3710820104',
                4: '荣成市人民医院预防接种门诊 3710828801',
                5: '俚岛中心卫生院预防接种门诊 3710820301',
                6: '俚岛镇马道卫生院接种门诊 3710820302',
                7: '荣成市俚岛镇马道卫生院狂犬病暴露处置门诊 3710820381',
                8: '荣成市俚岛中心卫生院狂犬病暴露处置门诊 3710820382',
                9: '第三人民医院预防接种门诊 3710820501',
                10: '成山镇龙须岛卫生院预防接种门诊 3710820502',
                11: '荣成市成山镇龙须岛卫生院狂犬病暴露处置门诊 3710820581',
                12: '荣成市第三人民医院狂犬病暴露处置门诊 3710820582',
                13: '埠柳中心卫生院预防接种门诊 3710820701',
                14: '荣成市埠柳中心卫生院狂犬病暴露处置门诊 3710820781',
                15: '港西镇卫生院预防接种门诊 3710820801',
                16: '荣成市港西镇卫生院狂犬病暴露处置门诊 3710820881',
                17: '整骨医院预防接种门诊 3710820901',
                18: '荣成市整骨医院狂犬病暴露处置门诊 3710820981',
                19: '夏庄卫生院预防接种门诊 3710821001',
                20: '荣成市夏庄镇卫生院狂犬病暴露处置门诊 3710821081',
                21: '荫子镇卫生院预防接种门诊 3710821101',
                22: '荣成市荫子镇卫生院狂犬病暴露处置门诊 3710821181',
                23: '大疃镇卫生院预防接种门诊 3710821301',
                24: '荣成市大疃镇卫生院狂犬病暴露处置门诊 3710821381',
                25: '上庄中心卫生院预防接种门诊 3710821401',
                26: '荣成市上庄中心卫生院狂犬病暴露处置门诊 3710821481',
                27: '人和中心卫生院预防接种门诊 3710821801',
                28: '人和镇靖海卫生院预防接种门诊 3710821802',
                29: '荣成市人和中心卫生院狂犬病暴露处置门诊 3710821881',
                30: '荣成市人和镇靖海卫生院狂犬病暴露处置门诊 3710821882',
                31: '荣成市石岛社区卫生服务中心接种门诊 3710821901',
                32: '王连卫生院预防接种门诊 3710823101',
                33: '宁津卫生院预防接种门诊 3710823201',
                34: '东山中心卫生院预防接种门诊 3710823301',
                35: '斥山卫生院预防接种门诊 3710823401',
                36: '滕家中心卫生院预防接种门诊 3710822501',
                37: '荣成市滕家中心卫生院狂犬病暴露处置门诊 3710822581',
                38: '虎山镇黄山卫生院预防接种门诊 3710822701',
                39: '虎山镇卫生院预防接种门诊 3710822702',
                40: '荣成市虎山镇黄山卫生院狂犬病暴露处置门诊 3710822781',
                41: '荣成市虎山镇卫生院狂犬病暴露处置门诊 3710822782',
                42: '东城区社区卫生服务中心预防接种门诊 3710822801',
                43: '荣成市中医院成人预防接种门诊 3710822804',
                44: '荣成市人民医院狂犬病暴露处置门诊 3710822881',
                45: '荣成市中医院狂犬病暴露处置门诊 3710822882',
                46: '荣成市康宁医院狂犬病暴露处置门诊 3710822981',
                47: '荣成市寻山街道卫生院接种点 3710823041',
                48: '荣成市寻山街道卫生院狂犬病暴露处置门诊 3710823081',
                49: '荣成市崂山街道卫生院狂犬病暴露处置门诊 3710823181',
                50: '荣成市宁津街道卫生院狂犬病暴露处置门诊 3710823281',
                51: '荣成市石岛社区卫生服务中心狂犬病暴露处置门诊 3710823381',
                52: '荣成市东山街道中心卫生院桃园预防接种门诊 3710823403',
                53: '荣成市王连街道卫生院狂犬病暴露处置门诊 3710823581',
                54: '荣成市东山街道中心卫生院狂犬病暴露处置门诊 3710823681',
                55: '荣成市石岛人民医院接种点 3710823741',
                56: '斥山街道卫生院狂犬病暴露处置门诊 3710823781',
                57: '荣成市石岛人民医院狂犬病暴露处置门诊 3710823782'
            },
            '对应单位': {
                0: '寻山',
                1: '崂山',
                2: '中医院',
                3: '康宁',
                4: '一院',
                5: '俚岛',
                6: '马道',
                7: '马道',
                8: '俚岛',
                9: '成山',
                10: '龙须',
                11: '龙须',
                12: '成山',
                13: '埠柳',
                14: '埠柳',
                15: '港西',
                16: '港西',
                17: '整骨',
                18: '整骨',
                19: '夏庄',
                20: '夏庄',
                21: '荫子',
                22: '荫子',
                23: '大疃',
                24: '大疃',
                25: '上庄',
                26: '上庄',
                27: '人和',
                28: '靖海',
                29: '人和',
                30: '靖海',
                31: '港湾',
                32: '王连',
                33: '宁津',
                34: '东山',
                35: '斥山',
                36: '滕家',
                37: '滕家',
                38: '黄山',
                39: '虎山',
                40: '黄山',
                41: '虎山',
                42: '东城',
                43: '中医院',
                44: '一院',
                45: '中医院',
                46: '康宁',
                47: '方舱',
                48: '寻山',
                49: '崂山',
                50: '宁津',
                51: '港湾',
                52: '桃园',
                53: '王连',
                54: '东山',
                55: '二院',
                56: '斥山',
                57: '二院'
            },
            '对应镇街': {
                0: '寻山街道',
                1: '崂山街道',
                2: '崖头街道',
                3: '城西街道',
                4: '崖头街道',
                5: '俚岛镇',
                6: '俚岛镇',
                7: '俚岛镇',
                8: '俚岛镇',
                9: '成山镇',
                10: '成山镇',
                11: '成山镇',
                12: '成山镇',
                13: '埠柳镇',
                14: '埠柳镇',
                15: '港西镇',
                16: '港西镇',
                17: '崖西镇',
                18: '崖西镇',
                19: '夏庄镇',
                20: '夏庄镇',
                21: '荫子镇',
                22: '荫子镇',
                23: '大疃镇',
                24: '大疃镇',
                25: '上庄镇',
                26: '上庄镇',
                27: '人和镇',
                28: '人和镇',
                29: '人和镇',
                30: '人和镇',
                31: '港湾街道',
                32: '王连街道',
                33: '宁津街道',
                34: '东山街道',
                35: '斥山街道',
                36: '滕家镇',
                37: '滕家镇',
                38: '虎山镇',
                39: '虎山镇',
                40: '虎山镇',
                41: '虎山镇',
                42: '崖头街道',
                43: '崖头街道',
                44: '崖头街道',
                45: '崖头街道',
                46: '城西街道',
                47: '崖头街道',
                48: '寻山街道',
                49: '崂山街道',
                50: '宁津街道',
                51: '港湾街道',
                52: '桃园街道',
                53: '王连街道',
                54: '东山街道',
                55: '斥山街道',
                56: '斥山街道',
                57: '斥山街道'
            }
        }

    def table_mapping(self, name, tables):
        if name[0] == '1':
            table = tables[2:60].copy().reset_index(
                drop=True)  # 重置index以便对应镇街Series的index
            table = table.iloc[:, [3, 5, 7, 1]]
            if name == '1-1-1':
                table = table.astype(int)
            else:
                table = table.astype(float)
        elif name[0] == '2':
            table = tables[2:60].copy().reset_index(
                drop=True)  # 重置index以便对应镇街Series的index
            table = table.iloc[:, [4, 8]]  # 取vero细胞第三剂累计和腺病毒第二剂累计
            table = table.astype(float)
        else:
            table = tables[1:59].copy().reset_index(
                drop=True)  # 表格索引稍有变化，重置index以便对应镇街Series的index
            table = table.iloc[:, [1, 2]].astype(float)  # 应种，实种

        return table

    def shift(self, name, tables):
        # table = tables[2:60].copy().reset_index(drop=True) # 重置index以便对应镇街Series的index
        # table = table.drop([0],axis=1) # 去掉中文列，其余列转换为float才能计算
        # table = table.astype(int) # 做个判断函数
        table = self.table_mapping(name, tables)
        table['zhenjie'] = pd.DataFrame(
            self.zhenjie_mapping)['对应镇街']  # 添加对应合并列
        # table
        table = table.groupby(['zhenjie']).sum(0)  # 计算
        # print(table)
        return table


# 制作第二版第一针实重调整表
class TiaoZheng():
    def __init__(self):
        # 60岁以上人群实种一针需调整
        self.t60_shi = pd.DataFrame({
            'zhenjie': [
                '宁津街道', '港湾街道', '桃园街道', '东山街道', '斥山街道', '崖头街道', '寻山街道', '崂山街道',
                '俚岛镇', '成山镇', '港西镇', '虎山镇', '人和镇', '王连街道', '城西街道', '埠柳镇',
                '夏庄镇', '崖西镇', '荫子镇', '滕家镇', '大疃镇', '上庄镇'
            ],
            '60': [
                0, 0, 0, 0, 0, 1175, 0, -351, 0, -278, -183, 0, 0, 0, 0, -141,
                -104, 0, -54, -64, 0, 0
            ],
            '69': [
                0, 0, 0, 0, 0, -100, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 17, 0, 0,
                68, 0, 15
            ],
            '79': [
                0, 0, 0, 0, 0, 187, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, -3, 0, 0,
                -78, 0, 0
            ],
            '80': [
                0, 0, 0, 0, 0, 1175, 0, -351, 0, -278, -183, 0, 0, 0, 0, -141,
                -104, 0, -54, -64, 0, 0
            ]
        }).set_index('zhenjie').sort_index()
        # 7普人口数据
        self.t60_ying = pd.DataFrame({
            'zhenjie': [
                '宁津街道', '港湾街道', '桃园街道', '东山街道', '斥山街道', '崖头街道', '寻山街道', '崂山街道',
                '俚岛镇', '成山镇', '港西镇', '虎山镇', '人和镇', '王连街道', '城西街道', '埠柳镇',
                '夏庄镇', '崖西镇', '荫子镇', '滕家镇', '大疃镇', '上庄镇'
            ],
            '60': [
                7971, 13674, 4690, 6296, 12798, 33657, 7258, 7448, 12649,
                13011, 5641, 13377, 20073, 6047, 5874, 8299, 4107, 6271, 4754,
                10040, 6039, 8422
            ],
            '69': [
                4151, 8141, 2652, 3309, 7578, 19856, 3977, 3966, 6727, 7196,
                3157, 7069, 11280, 3323, 3259, 4577, 2367, 3438, 2606, 5515,
                3404, 4668
            ],
            '79': [
                2649, 3944, 1397, 2120, 3660, 9737, 2368, 2235, 4272, 4197,
                1740, 4028, 5829, 1924, 1722, 2517, 1220, 1865, 1441, 3023,
                1778, 2464
            ],
            '80': [
                1171, 1589, 641, 867, 1560, 4064, 913, 1247, 1650, 1618, 744,
                2280, 2964, 800, 893, 1205, 520, 968, 707, 1502, 857, 1290
            ]
        }).set_index('zhenjie').sort_index()


def resulting(df, pst_tables):
    """处理合并镇街后的表格，出结果表
    Parameters
    ----------
    df : dict of dfs : {'1-1-1': shifted_table}
    """
    # 第一版上周接种情况
    df_1 = df['1-1-1'].copy()
    df_1 = df_1.sort_values(by=[1], ascending=False)
    df_1_sum = df_1.sum()
    df_1.loc['合计'] = df_1_sum
    df1 = df_1.reset_index()
    # 第二版首针60岁以上接种情况
    tiaozheng_60 = TiaoZheng()  # 实例化60岁以上第一针实种调整
    df_2 = df['1-2-1'].copy()[3].to_frame(name='60')
    df_3 = df['1-2-2'].copy()[3].to_frame(name='69')
    df_4 = df['1-2-3'].copy()[3].to_frame(name='79')
    df_5 = df['1-2-4'].copy()[3].to_frame(name='80')
    df_2_ = pd.concat([df_2, df_3, df_4, df_5], axis=1)
    df_2_ = df_2_ + tiaozheng_60.t60_shi  # 调整实种一针
    df_7pu = tiaozheng_60.t60_ying
    df2_sum = (100 * df_2_.sum() / df_7pu.sum()).round(2).to_frame(name='合计')
    df2 = df_2_ / df_7pu
    df2 = (100 * df2).round(2)
    df_2 = pd.concat(
        [df2['60'].sort_values(ascending=False),
         df2_sum.loc['60']]).reset_index()
    df_3 = pd.concat(
        [df2['69'].sort_values(ascending=False),
         df2_sum.loc['69']]).reset_index()
    df_4 = pd.concat(
        [df2['79'].sort_values(ascending=False),
         df2_sum.loc['79']]).reset_index()
    df_5 = pd.concat(
        [df2['80'].sort_values(ascending=False),
         df2_sum.loc['80']]).reset_index()
    df2 = pd.concat([df_2, df_3, df_4, df_5], axis=1)
    # 第三版18岁以上加强
    df_3_1 = df['1-3-1-1'].copy()[5]  # 第二剂18岁以上半年前接种
    df_3_2 = df['1-3-1-2'].copy()[5]  # 康熙诺腺病毒、北京智飞绿竹重组蛋白、安徽志飞重组蛋白第二剂截止周天
    df_3_3 = df['1-3-1-3'].copy()[3]  # 康熙诺腺病毒第一剂截止周天
    df_3_ying = (df_3_1 + df_3_3 - df_3_2)
    df_3_shi = df['2-3-1-1'].copy().sum(1)
    df3_ = (100 * df_3_shi / df_3_ying).round(2).sort_values(ascending=False)
    df_3_sum = (100 * df_3_shi.sum() / df_3_ying.sum()).round(2)
    df3_['合计'] = df_3_sum
    df3 = df3_.reset_index()
    # 第三版导出指标18岁以上加强、60岁以上全程、60岁以上加强
    df_4 = df['3-3-2'].copy()  # 18岁以上按归属地计算加免接种率
    df4_ = (100 * df_4[2] / df_4[1]).round(2).sort_values(ascending=False)
    df4_['合计'] = (100 * df_4[2].sum() / df_4[1].sum()).round(2)
    df4 = df4_.reset_index()
    df_5 = df['4-3-3'].copy()  # 60岁以上全程接种率
    df5_ = (100 * df_5[2] / df_5[1]).round(2).sort_values(ascending=False)
    df5_['合计'] = (100 * df_5[2].sum() / df_5[1].sum()).round(2)
    df5 = df5_.reset_index()
    df_6 = df['3-3-4'].copy()  # 60岁以上全程接种率
    df6_ = (100 * df_6[2] / df_6[1]).round(2).sort_values(ascending=False)
    df6_['合计'] = (100 * df_6[2].sum() / df_6[1].sum()).round(2)
    df6 = df6_.reset_index()
    result_table = pd.concat([df1, df2, df3, df4, df5, df6], axis=1)
    result_table.to_csv(
        f'./dfiles/{pst_tables.date}/{pst_tables.weekago} - {pst_tables.date} - result_tables.csv',
        encoding='utf-8',
        index=False)


def open_file(file_name):
    """创建文件夹
    Parameters
    ----------
    file_name : 日期字符串
    """
    if os.path.exists('./dfiles/' + file_name):
        pass
    else:
        os.mkdir('./dfiles/' + file_name)


def df_pkl(date: str):
    """临时缓冲文件，避免网络不好
    Parameters
    ----------
    date : pst_tables.date ，用作保存路径
    """
    if not os.path.exists(f'./dfiles/{date}/df.pkl'):
        pkl_file = open(f'./dfiles/{date}/df.pkl', 'wb')
        df = {}
        pickle.dump(df, pkl_file)
        pkl_file.close()
    else:  # 读取备用df.pkl
        load_pkl = open(f'./dfiles/{date}/df.pkl', 'rb')
        df = pickle.load(load_pkl)
        load_pkl.close()

    return df


def main():
    # 1.首先连接VPN。
    # 启动selenium登录省网
    denglu_link = 'http://10.60.0.52:83/sysmanage_sd/systems.jsp'
    cruser = webdriver.Firefox()
    cruser.get(denglu_link)
    time.sleep(random.randint(1, 2))
    cruser.find_element_by_id('username').send_keys('******')
    cruser.find_element_by_id('password').send_keys('******')
    cruser.find_element_by_id('vaildCode').send_keys('')
    time.sleep(random.randint(1, 2))
    WebDriverWait(cruser,
                  60).until(EC.presence_of_all_elements_located(
                      (By.ID, 'app')))
    # print('1.登录成功时的cookies')
    # print(cruser.get_cookies())
    name = cruser.find_elements_by_xpath(
        "//div[@class='centerBox']//a[@title='预防接种信息管理']/img")[0]
    name.click()
    time.sleep(random.randint(1, 2))
    # print('2.点击跳转后的cookies')
    # print(cruser.get_cookies())
    # 2.到达jsp页面 获得cookies转为requests.session直接post请求
    requests_session = requests.Session()
    requests_session.headers.update({
        'Accept':
        'application/json, text/javascript, */*; q=0.01',
        'Accept-Encoding':
        'gzip, deflate',
        'Accept-Language':
        'zh-CN,zh;q=0.9,en;q=0.8,en-GB;q=0.7,en-US;q=0.6',
        'Cache-Control':
        'max-age=0',
        'Connection':
        'keep-alive',
        'Content-Type':
        'application/x-www-form-urlencoded',
        'Host':
        '10.60.0.16:83',
        'Origin':
        'http://10.60.0.16:83',
        'Referer':
        'http://10.60.0.16:83/ShanDong/gjdc/xgRptArea_query.action',
        'Upgrade-Insecure-Requests':
        '1',
        'user-agent':
        'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/103.0.5060.53 Safari/537.36 Edg/103.0.1264.37'
    })
    sd_cookie = cruser.get_cookies()[0]['value']
    requests_session.headers.update(
        {'cookie': f'JSESSIONID_shandong={sd_cookie}'})

    # 利用 PostTables类 生成post表单
    # InputBox 接收半年前日期
    # print('请拿起你的小手，在键盘上输入半年前的日期，月底对应月底，例如 2022-03-28')
    # date_half_year_ago = input('输入半年前的日期，月底对应月底，例如 2022-03-28')
    # ! 写个函数自动判断了
    pst_tables = PostTables()

    # 备份原始表 checked ok
    open_file(pst_tables.date)
    df = df_pkl(pst_tables.date)
    for name, pst in pst_tables.gen_tables.items():
        if name not in df:
            target_url = pst_tables.cmodid_to_url[pst['cModId']]
            resp = requests_session.post(target_url, data=pst)
            time.sleep(random.randint(1, 2))
            resp.encoding = resp.apparent_encoding
            print(
                f'name = {name} , encoding = {resp.encoding} , status_code={resp.status_code}'
            )
            # print(resp.text)
            try:
                tables = pd.read_html(resp.text, encoding='utf-8')[-1]
                # 接着处理表格，转换对应镇街
                tables = TablesShift().shift(name=name, tables=tables)
                df.update({name: tables})
                # 备份系统原始表
                tables.to_csv(f'./dfiles/{pst_tables.date}/{name}.csv',
                              encoding='utf-8',
                              index=False)
                print(f'完成！ 保存地址 ./dfiles/{pst_tables.date}/{name}.csv')
            except:
                print(
                    f'检查一下post表单，看看是不是有问题 : name = {name} , status_code={resp.status_code}'
                )
            continue
        else:
            print(f'表{name}已经爬取')
        continue

    with open(f'./dfiles/{pst_tables.date}/df.pkl', 'wb') as pkl_file:
        pickle.dump(df, pkl_file)

    resulting(df, pst_tables)
    print('------ 全部完成，请进入日期文件夹选择 result.csv 复制表格 ------')
    print('--- 注意：1. 70-79岁接种覆盖率 崖头街道 +1% 92.36% ---')
    print('--- 注意：2. 80岁以上接种覆盖率 虎山镇 调整 85.00%以上 ---')

    cruser.quit()


if __name__ == '__main__':
    main()